In [5]:
import pandas as pd
import os
data_path = "../data/running/"

<p>"datetime: date of the running activity;

athlete: a computer-generated ID for the athlete (integer);

distance: distance of running (floating-point number, in kilometers);

duration: duration of running (floating-point number, in minutes);

gender: gender (string 'M' of 'F');

age_group: age interval (one of the strings '18 - 34', '35 - 54', or '55 +');

country: country of origin of the athlete (string);

major: marathon(s) and year(s) the athlete ran (comma-separated list of strings)."</p>

For convenience, we created files with the athletes' activities data sampled at different frequencies: day 'd', week 'w', month 'm', and quarter 'q' (i.e., there are files with the distance and duration of running accumulated at each day, week, month, and quarter) for each year, 2019 and 2020. Accordingly, the files are named 'run_ww_yyyy_f.parquet', where 'yyyy' is '2019' or '2020' and 'f' is 'd', 'w', 'm' or 'q' (without quotes).

In [3]:
list_files = [f for f in os.listdir(data_path) if (not f.endswith(".zip")) & (f.startswith("run_"))]
for file in list_files:
    df = pd.read_csv(data_path+file)
    print(f"{file} nb rows : {len(df)}")

run_ww_2019_m.csv nb rows : 436944
run_ww_2020_m.csv nb rows : 436944
run_ww_2020_w.csv nb rows : 1893424
run_ww_2020_q.csv nb rows : 145648
run_ww_2020_d.csv nb rows : 13326792
run_ww_2019_w.csv nb rows : 1893424
run_ww_2019_q.csv nb rows : 145648
run_ww_2019_d.csv nb rows : 13290380


Check for Nan

In [15]:
data_path = "./data/running/"
list_files = [f for f in os.listdir(data_path) if (not f.endswith(".zip")) & (f.startswith("run_"))]
for file in list_files:
    df = pd.read_csv(data_path+file)
    print(f"{file} nb rows : {len(df)}")
    print(df.isnull().sum())
    print()

run_ww_2019_m.csv nb rows : 436944
Unnamed: 0       0
datetime         0
athlete          0
distance         0
duration         0
gender           0
age_group        0
country       3948
major            0
dtype: int64

run_ww_2020_m.csv nb rows : 436944
Unnamed: 0       0
datetime         0
athlete          0
distance         0
duration         0
gender           0
age_group        0
country       3948
major            0
dtype: int64

run_ww_2020_w.csv nb rows : 1893424
Unnamed: 0        0
datetime          0
athlete           0
distance          0
duration          0
gender            0
age_group         0
country       17108
major             0
dtype: int64

run_ww_2020_q.csv nb rows : 145648
Unnamed: 0       0
datetime         0
athlete          0
distance         0
duration         0
gender           0
age_group        0
country       1316
major            0
dtype: int64

run_ww_2020_d.csv nb rows : 13326792
Unnamed: 0         0
datetime           0
athlete            0
distance  

In [6]:
df = pd.read_csv(data_path+"run_ww_2019_d.csv", index_col="Unnamed: 0")

In [12]:
df.loc[(df['athlete'] == 11028) & (df['distance'] != 0), ]

,datetime,athlete,distance,duration,gender,age_group,country,major
10652,2019-01-01,11028,4.83,26.116667,M,35 - 54,United States,BOSTON 2019
47064,2019-01-02,11028,6.16,30.350000,M,35 - 54,United States,BOSTON 2019
83476,2019-01-03,11028,9.16,43.783333,M,35 - 54,United States,BOSTON 2019
119888,2019-01-04,11028,9.39,43.466667,M,35 - 54,United States,BOSTON 2019
156300,2019-01-05,11028,10.78,50.800000,M,35 - 54,United States,BOSTON 2019
...,...,...,...,...,...,...,...,...
13118972,2019-12-27,11028,6.45,30.600000,M,35 - 54,United States,BOSTON 2019
13155384,2019-12-28,11028,4.96,27.433333,M,35 - 54,United States,BOSTON 2019
13191796,2019-12-29,11028,5.42,30.300000,M,35 - 54,United States,BOSTON 2019
13228208,2019-12-30,11028,4.93,29.166667,M,35 - 54,United States,BOSTON 2019


In [24]:
df_population = pd.read_csv("./data/population/API_SP.POP.TOTL_DS2_en_csv_v2_87.csv", sep=',', skiprows=4)

data processing

In [29]:
if ('Indicator Name' in df_population.columns) & ('Indicator Code' in df_population.columns):
    df_population.drop(columns=['Indicator Name', 'Indicator Code'], inplace=True)
df_population.rename(columns={'Country Name': 'country_name', 'Country Code': 'country_code'}, inplace=True)
df_pop_processed = pd.melt(df_population, id_vars=['country_name', 'country_code'], value_vars=df_population.columns[2:], var_name='year', value_name="population")
df_pop_processed.dropna(inplace=True)
df_pop_2019_2020 = df_pop_processed.loc[df_pop_processed['year'].isin(["2019", "2020"])]
df_pop_2019_2020.to_csv("./data/population/population_processed.csv", index=False, sep=",")

World records running

In [2]:
distance_dict = {
    '100 Meters': 0.1,
    '200 Meters': 0.2,
    '400 Meters': 0.4,
    '800 Meters': 0.8,
    '1000 Meters': 1.0,
    '1500 Meters': 1.5,
    '2000 Meters': 2.0,
    '3000 Meters': 3.0,
    '5000 Meters': 5.0,
    '5 Kilometers': 5.0,
    '10,000 Meters': 10.0,
    '10 Kilometers': 10.0,
    'Half Marathon': 21.0975,  # 21.0975 km (official half marathon distance)
    'Marathon': 42.195,  # 42.195 km (official marathon distance)
    '50 Kilometers': 50.0,
    '100 Kilometers': 100.0,
    'Half Marathon (Mixed Gender Race)': 21.0975,
    '10 Kilometers (Mixed Gender Race)': 10.0,
    '50 Kilometers (Mixed Gender Race)': 50.0,
    'Marathon (Mixed Gender Race)': 42.195,
    '5 Kilometers (Mixed Gender Race)': 5.0
}

In [3]:
# Function to convert duration to minutes
def convert_to_minutes(duration):
    # Split the time based on ":" if present
    if ':' in duration:
        parts = duration.split(':')
        # Handle different formats
        if len(parts) == 2:  # minutes:seconds format
            minutes = float(parts[0]) + float(parts[1]) / 60
        elif len(parts) == 3:  # hours:minutes:seconds format
            minutes = float(parts[0]) * 60 + float(parts[1]) + float(parts[2]) / 60
    else:  # For times like "9.58" (just seconds)
        minutes = float(duration) / 60
    return minutes

In [4]:
def process_running_world_record(df_, gender):
	df_running = df_
	df_running = df_running.rename(columns={
		"Event" : "event", 
		"World Record": "world_record", 
		"Competitor": "athlete", 
		"Age At Time of Record": "athlete_age", 
		"Country": "country", 
		"Venue Location": "venu_location",	
		"Date of Record": "record_date",
		"Record Holder": "athlete",
		"Age at Time of Record": "athlete_age"})
	filters = (~df_running["event"].str.contains("Relay")) & \
		(~df_running["event"].str.contains("Hurdle")) & \
		(~df_running["event"].str.contains("Walk")) & \
		(~df_running["event"].str.contains("Decathlon")) & \
		(~df_running["event"].str.contains("Steeplechase")) & \
		(~df_running["event"].str.contains("Miles")) & \
		(~df_running["event"].str.contains("Mile")) & \
		(~df_running["event"].str.contains("Hour")) & \
		(~df_running["event"].str.contains("Only")) & \
		(~df_running["event"].str.contains("Women’s")) & \
		(~df_running["event"].str.contains("Heptathlon")) & \
		(~df_running["event"].str.contains("10 Kilometers (Mixed Gender Race)"))
	df_running = df_running.loc[filters]
	df_running.loc[df_running['world_record'] == '14:00:2', 'world_record'] = '14:00.2'
	df_running.loc[df_running['world_record'] == '57:31:00', 'world_record'] = '57:31.00'
	df_running['distance'] = df_running.apply(lambda x : distance_dict[x['event']], axis=1)
	df_running['duration'] = df_running['world_record'].apply(convert_to_minutes)
	df_running['gender'] = gender
	df_running['event'] = df_running['event'].str.replace(" (Mixed Gender Race)", "")

	
	return df_running

In [5]:
from bs4 import BeautifulSoup
import pandas as pd

html_content = """[<table><tbody><tr><td><strong>Event</strong></td><td><strong>World Record</strong></td><td><strong>Competitor</strong></td><td><strong>Age At Time of Record</strong></td><td><strong>Country</strong></td><td><strong>Venue Location</strong></td><td><strong>Date of Record</strong></td></tr><tr><td>100 Meters</td><td>9.58</td><td>Usain Bolt</td><td>22</td><td>Jamaica</td><td>Berlin, Germany</td><td>8/16/09</td></tr><tr><td>200 Meters</td><td>19.19</td><td>Usain Bolt</td><td>22</td><td>Jamaica</td><td>Berlin, Germany</td><td>8/20/09</td></tr><tr><td>400 Meters</td><td>43.03</td><td>Wayde Van Niekerk</td><td>24</td><td>South Africa</td><td>Rio de Janeiro, Brazil</td><td>8/14/16</td></tr><tr><td>800 Meters</td><td>01:40.9</td><td>David Rudisha</td><td>23</td><td>Kenya</td><td>London, Great Britain</td><td>8/9/12</td></tr><tr><td>1000 Meters</td><td>02:12.0</td><td>Noah Ngeny</td><td>20</td><td>Kenya</td><td>Rieti, Italy</td><td>9/5/99</td></tr><tr><td>1500 Meters</td><td>03:26.0</td><td>Hicham El Guerrouj</td><td>23</td><td>Morocco</td><td>Rome, Italy</td><td>7/14/98</td></tr><tr><td>One Mile</td><td>03:43.1</td><td>Hicham El Guerrouj</td><td>24</td><td>Morocco</td><td>Rome, Italy</td><td>7/7/99</td></tr><tr><td>2000 Meters</td><td>4:43.1</td><td>Jakob Ingebrigsten</td><td>22</td><td>Norway</td><td>Brussels, Belgium</td><td>8/9/23</td></tr><tr><td>3000 Meters</td><td>07:17.6</td><td>Jakob Ingebrigsten</td><td>24</td><td>Norway</td><td>Chorzów, Poland</td><td>25/8/24</td></tr><tr><td>5000 Meters</td><td>12:35.4</td><td>Joshua Cheptegei</td><td>23</td><td>Uganda</td><td>Monaco, Monaco</td><td>8/14/20</td></tr><tr><td>5 Kilometers</td><td>12:49</td><td>Berihu Aregawi</td><td>20</td><td>Ethiopia</td><td>Barcelona, Spain</td><td>12/31/21</td></tr><tr><td>10,000 Meters</td><td>26:11.0</td><td>Joshua Cheptegei</td><td>24</td><td>Uganda</td><td>Valencia, Spain</td><td>10/7/20</td></tr><tr><td>10 Kilometers</td><td>26:24:00</td><td>Rhonex Kipruto</td><td>20</td><td>Kenya</td><td>Valencia, Spain</td><td>1/12/20</td></tr><tr><td>One Hour</td><td>21,330 meters</td><td>Mo Farah</td><td>37</td><td>Great Britain</td><td>Bruxelles, Belgium</td><td>9/4/20</td></tr><tr><td>Half Marathon</td><td>57:31:00</td><td>Jacob Kiplimo</td><td>21</td><td>Uganda</td><td>Lisbon, Portugal</td><td>11/21/21</td></tr><tr><td>Marathon</td><td>2:00:35</td><td>Kelvin Kiptum</td><td>24</td><td>Kenya</td><td>Chicago, IL, USA</td><td>8/10/23</td></tr><tr><td>50 Kilometers</td><td>2:38:43</td><td>CJ Albertson</td><td>29</td><td>USA</td><td>San Francisco, CA, USA</td><td>8/10/22</td></tr><tr><td>100 Kilometers</td><td>6:05:41</td><td>Aleksandr Sorokin</td><td>40</td><td>Lithuania</td><td>Bedford, United Kingdom</td><td>4/23/22</td></tr><tr><td>100 Miles</td><td>10:51:39</td><td>Aleksandr Sorokin</td><td>40</td><td>Lithuania</td><td>Tel Aviv, Israel</td><td>1/6/22</td></tr><tr><td>3000 Meters Steeplechase</td><td>7:52.11</td><td>Lamecha Girma</td><td>22</td><td>Ethiopia</td><td>Paris, france</td><td>9/6/23</td></tr><tr><td>110 Meters Hurdles</td><td>12.8</td><td>Aries Merritt</td><td>27</td><td>USA</td><td>Bruxelles, Belgium</td><td>9/7/12</td></tr><tr><td>400 Meters Hurdles</td><td>45.94</td><td>Karsten Warholm</td><td>25</td><td>Norway</td><td>Tokyo, Japan</td><td>8/3/21</td></tr><tr><td>Decathlon</td><td>9126 points</td><td>Kevin Mayer</td><td>26</td><td>France</td><td>Talence, France</td><td>9/16/18</td></tr><tr><td>20,000 Meters Race Walk</td><td>1:17:25.6</td><td>Bernardo Segura</td><td>24</td><td>Mexico</td><td>Fana, Norway</td><td>5/7/94</td></tr><tr><td>20 Kilometers Race Walk</td><td>1:16:36</td><td>Yusuke Suzuki</td><td>27</td><td>Japan</td><td>Nomi, Japan</td><td>3/15/15</td></tr><tr><td>30,000 Meters Race Walk</td><td>2:01:44.1</td><td>Maurizio Damilano</td><td>35</td><td>Italy</td><td>Cuneo, Italy</td><td>10/3/92</td></tr><tr><td>50,000 Meters Race Walk</td><td>3:35:27.2</td><td>Yohann Diniz</td><td>33</td><td>France</td><td>Reims, France</td><td>3/12/11</td></tr><tr><td>50 Kilometers Race Walk</td><td>3:32:33</td><td>Yohann Diniz</td><td>36</td><td>France</td><td>Zürich, Switzerland</td><td>8/15/14</td></tr><tr><td>4×100 Meters Relay</td><td>36.84</td><td>Jamaica</td><td></td><td>Jamaica</td><td>London, Great Britain</td><td>8/11/12</td></tr><tr><td>4×200 Meters Relay</td><td>01:18.6</td><td>Jamaica</td><td></td><td>Jamaica</td><td>Nassau, Bahamas</td><td>5/24/14</td></tr><tr><td>4×400 Meters Relay</td><td>02:54.3</td><td>United States</td><td></td><td>USA</td><td>Stuttgart, Germany</td><td>8/22/93</td></tr><tr><td>4×800 Meters Relay</td><td>07:02.4</td><td>Kenya</td><td></td><td>Kenya</td><td>Bruxelles, Belgium</td><td>8/25/06</td></tr><tr><td>4×1500 Meters Relay</td><td>14:22.2</td><td>Kenya</td><td></td><td>Kenya</td><td>Nassau, Bahamas</td><td>5/25/14</td></tr><tr><td>Road Relay</td><td>1:57:06</td><td>Kenya</td><td></td><td>Kenya</td><td>Chiba, Japan</td><td>11/23/05</td></tr><tr><td>Distance Medley Relay</td><td>09:15.5</td><td>United States</td><td></td><td>USA</td><td>Nassau, Bahamas</td><td>5/3/15</td></tr></tbody></table>]"""


# Step 1: Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Step 2: Find the table
table = soup.find('table')

# Step 3: Extract headers (column names)
headers = []
for th in table.find_all('strong'):
    headers.append(th.get_text(strip=True))

# Step 4: Extract rows (data) from the table
rows = []
for tr in table.find_all('tr')[1:]:  # Skip the header row
    cols = tr.find_all('td')
    if len(cols) > 0:
        row = [col.get_text(strip=True) for col in cols]
        rows.append(row)

# Step 5: Create a DataFrame from the extracted data
df_male = pd.DataFrame(rows, columns=headers)

In [6]:
html_content = """[<table><tbody><tr><td><strong>Event</strong></td><td><strong>World Record</strong></td><td><strong>Record Holder</strong></td><td><strong>Age at Time of Record</strong></td><td><strong>Country</strong></td><td><strong>Venue Location</strong></td><td><strong>Date of Record</strong></td></tr><tr><td>100 Meters</td><td>10.49</td><td>Florence Griffith-Joyner</td><td>28</td><td>USA</td><td>Indianapolis, IN, USA</td><td>7/16/88</td></tr><tr><td>200 Meters</td><td>21.34</td><td>Florence Griffith-Joyner</td><td>28</td><td>USA</td><td>Seoul, South Korea</td><td>9/29/88</td></tr><tr><td>400 Meters</td><td>47.6</td><td>Marita Koch</td><td>28</td><td>East Germany</td><td>Canberra, Australia</td><td>10/6/85</td></tr><tr><td>800 Meters</td><td>01:53.3</td><td>Jarmila Kratochvílová</td><td>32</td><td>Czechoslovakia</td><td>München, Germany</td><td>7/26/83</td></tr><tr><td>1000 Meters</td><td>02:29.0</td><td>Svetlana Masterkova</td><td>28</td><td>Russia</td><td>Bruxelles, Belgium</td><td>8/23/96</td></tr><tr><td>1500 Meters</td><td>3:49.1</td><td>Faith Kipyegon</td><td>29</td><td>Kenya</td><td>Firenze, Italy</td><td>6/2/23</td></tr><tr><td>One Mile</td><td>4:07.6</td><td>Faith Kipyegon</td><td>29</td><td>Kenya</td><td>Monaco</td><td>7/21/23</td></tr><tr><td>2000 Meters</td><td>05:19.7</td><td>Jessica Hull</td><td>28</td><td>Austrailia</td><td>Monaco</td><td>7/12/24</td></tr><tr><td>3000 Meters</td><td>08:06.1</td><td>Junxia Wang</td><td>20</td><td>China</td><td>Beijing, China</td><td>9/13/93</td></tr><tr><td>5000 Meters</td><td>14:00:2</td><td>Gudaf Tsegay</td><td>26</td><td>Ethiopia</td><td>Eugene, USA</td><td>9/17/23</td></tr><tr><td>5 Kilometers (Women’s Only Race)</td><td>14:13</td><td>Beatrice Chebet</td><td>23</td><td>Kenya</td><td>Barcelona, Spain</td><td>12/31/23</td></tr><tr><td>5 Kilometers (Mixed Gender Race)</td><td>14:13</td><td>Agnes Jebet Ngetich</td><td>23</td><td>Kenya</td><td>Valencia, Spain</td><td>1/14/24</td></tr><tr><td>10,000 Meters</td><td>29:01.0</td><td>Letesenbet Gidey</td><td>23</td><td>Ethiopia</td><td>Hengelo, Netherlands</td><td>6/8/21</td></tr><tr><td>10 Kilometers (Women’s Only Race)</td><td>30:01</td><td>Agnes Jebet Tirop</td><td>25</td><td>Kenya</td><td>Herzogenaurach, Germany</td><td>9/12/21</td></tr><tr><td>10 Kilometers (Mixed Gender Race)</td><td>28:46</td><td>Agnes Jebet Ngetich</td><td>23</td><td>Kenya</td><td>Valencua, Spain</td><td>1/14/24</td></tr><tr><td>One Hour</td><td>18,930 meters</td><td>Sifan Hassan</td><td>27</td><td>Netherlands</td><td>Bruxelles, Belgium</td><td>9/4/20</td></tr><tr><td>Half Marathon (Women’s Only Race)</td><td>1:05:16</td><td>Peres Jepchirchir</td><td>27</td><td>Kenya</td><td>Gdynia, Poland</td><td>10/17/20</td></tr><tr><td>Half Marathon (Mixed Gender Race)</td><td>1:02:52</td><td>Letesenbet Gidey</td><td>23</td><td>Ethiopia</td><td>Valencia, Spain</td><td>10/24/21</td></tr><tr><td>Marathon (Women’s Only Race)</td><td>2:16:16</td><td>Peres Jepchirchir</td><td>30</td><td>Kenya</td><td>London, Great Britain</td><td>4/21/24</td></tr><tr><td>Marathon (Mixed Gender Race)</td><td>2:09:56</td><td>Ruth Chepngetich</td><td>30</td><td>Kenya</td><td>Chicago, IL, USA</td><td>10/13/24</td></tr><tr><td>50 Kilometers (Women’s Only Race)</td><td>3:00:30</td><td>Emane Seifu</td><td></td><td>Ethiopia</td><td>Gqeberha, Russia</td><td>2/26/23</td></tr><tr><td>50 Kilometers (Mixed Gender Race)</td><td>2:59:54</td><td>Desiree Linden</td><td>37</td><td>USA</td><td>Dorena Lake, OR, USA</td><td>4/13/21</td></tr><tr><td>100 Kilometers</td><td>6:33:11</td><td>Tomoe Abe</td><td>28</td><td>Japan</td><td>Lake Saroma, Japan</td><td>6/25/00</td></tr><tr><td>3000 Meters Steeplechase</td><td>08:44.3</td><td>Beatrice Chepkoech</td><td>27</td><td>Kenya</td><td>Monaco, Monaco</td><td>7/20/18</td></tr><tr><td>100 Meters Hurdles</td><td>12.2</td><td>Kendra Harrison</td><td>23</td><td>USA</td><td>London, Great Britain</td><td>7/22/16</td></tr><tr><td>400 Meters Hurdles</td><td>50.37</td><td>Sydney McLaughlin-Levrone</td><td>25</td><td>USA</td><td>Paris, France</td><td>8/8/24</td></tr><tr><td>Heptathlon</td><td>7291</td><td>Jackie Joyner-Kersee</td><td>26</td><td>USA</td><td>Seoul, South Korea</td><td>9/24/88</td></tr><tr><td>Decathlon</td><td>8358 points</td><td>Austra Skujytė</td><td>25</td><td>Lithuania</td><td>Columbia, MO, USA</td><td>4/15/05</td></tr><tr><td>10,000 Meters Race Walk</td><td>41:56.2</td><td>Nadezhda Ryashkina</td><td>23</td><td>Soviet Union</td><td>Seattle, WA, USA</td><td>7/24/90</td></tr><tr><td>20,000 Meters Race Walk</td><td>1:26:52.3</td><td>Olimpiada Ivanova</td><td>31</td><td>Russia</td><td>Brisbane, Australia</td><td>9/6/01</td></tr><tr><td>20 Kilometers Race Walk</td><td>1:23:49</td><td>Jiayu Yang</td><td>25</td><td>China</td><td>Huangshan, China</td><td>3/20/21</td></tr><tr><td>50 Kilometers Race Walk</td><td>3:59:15</td><td>Hong Liu</td><td>31</td><td>China</td><td>Huangshan, China</td><td>3/9/19</td></tr><tr><td>4×100 Meters Relay</td><td>40.82</td><td>United States</td><td></td><td>USA</td><td>London, Great Britain</td><td>8/10/12</td></tr><tr><td>4×200 Meters Relay</td><td>01:27.5</td><td>United States Blue</td><td></td><td>USA</td><td>Philadelphia, PA, USA</td><td>4/29/00</td></tr><tr><td>4×400 Meters Relay</td><td>03:15.2</td><td>Soviet Union</td><td></td><td>Soviet Union</td><td>Seoul, South Korea</td><td>10/1/88</td></tr><tr><td>4×800 Meters Relay</td><td>07:50.2</td><td>Soviet Union</td><td></td><td>Soviet Union</td><td>Moskva, Soviet Union</td><td>8/5/84</td></tr><tr><td>4×1500 Meters Relay</td><td>16:27.0</td><td>Nike/Bowerman Track Club</td><td></td><td>USA</td><td>Portland, OR, USA</td><td>7/31/20</td></tr><tr><td>Road Relay</td><td>2:11:41</td><td>Pr Of China</td><td></td><td>China</td><td>Beijing, China</td><td>2/28/98</td></tr><tr><td>Distance Medley Relay</td><td>10:36.5</td><td>United States</td><td></td><td>USA</td><td>Nassau, Bahamas</td><td>5/2/15</td></tr></tbody></table>]"""

# Step 1: Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Step 2: Find the table
table = soup.find('table')

# Step 3: Extract headers (column names)
headers = []
for th in table.find_all('strong'):
    headers.append(th.get_text(strip=True))

# Step 4: Extract rows (data) from the table
rows = []
for tr in table.find_all('tr')[1:]:  # Skip the header row
    cols = tr.find_all('td')
    if len(cols) > 0:
        row = [col.get_text(strip=True) for col in cols]
        rows.append(row)

# Step 5: Create a DataFrame from the extracted data
df_female = pd.DataFrame(rows, columns=headers)

In [7]:
df_wr_male = process_running_world_record(df_male, 'M')
df_wr_female = process_running_world_record(df_female, 'F')
df_wr = pd.concat([df_wr_male, df_wr_female]).reset_index(drop=True)
df_wr = df_wr.loc[~df_wr['world_record'].isin(['14:13', '29:01.0', '12:49', '26:24:00'])]
df_wr.loc[df_wr['event'] == '10,000 Meters', 'event'] = "10 Kilometers"
df_wr.to_csv("../data/wr/running_wr.csv", index=False)

/tmp/ipykernel_70765/3756991036.py:24: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (~df_running["event"].str.contains("10 Kilometers (Mixed Gender Race)"))
/tmp/ipykernel_70765/3756991036.py:24: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (~df_running["event"].str.contains("10 Kilometers (Mixed Gender Race)"))
